In [32]:
import math
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy import stats
import pandas as pd
import geopandas as gpd
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.ensemble import IsolationForest
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import gaussian_kde, pearsonr
from matplotlib.ticker import FixedLocator

In [33]:
# Carregar shapefiles
gdf_model = gpd.read_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_model.shp")
gdf_log = gpd.read_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_model_log.shp")
gdf_sqrt = gpd.read_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_model_sqrt.shp")

# Converter para DataFrame puro (sem geometria)
df_model = pd.DataFrame(gdf_model.drop(columns='geometry'))
df_log = pd.DataFrame(gdf_log.drop(columns='geometry'))
df_sqrt = pd.DataFrame(gdf_sqrt.drop(columns='geometry'))

In [34]:
'''def remove_outliers_iqr_sync(*dfs, k=1.5):
    """
    Remove outliers de múltiplos dataframes de forma sincronizada usando IQR.
    
    Parâmetros:
        dfs: dataframes relacionados (mesmo número de linhas, diferentes transformações)
        k: fator IQR (default 1.5)
    
    Retorna:
        lista de dataframes limpos, com linhas sincronizadas
    """
    base_df = dfs[0]
    df_num = base_df.select_dtypes(include=np.number)
    Q1 = df_num.quantile(0.25)
    Q3 = df_num.quantile(0.75)
    IQR = Q3 - Q1
    
    # Máscara de linhas que não são outliers
    mask = ~((df_num < (Q1 - k*IQR)) | (df_num > (Q3 + k*IQR))).any(axis=1)
    
    # Aplica a mesma máscara a todos os dataframes
    return [df.loc[mask] for df in dfs]

# Uso
df_model, df_log, df_sqrt = remove_outliers_iqr_sync(df_model, df_log, df_sqrt)'''

'def remove_outliers_iqr_sync(*dfs, k=1.5):\n    """\n    Remove outliers de múltiplos dataframes de forma sincronizada usando IQR.\n\n    Parâmetros:\n        dfs: dataframes relacionados (mesmo número de linhas, diferentes transformações)\n        k: fator IQR (default 1.5)\n\n    Retorna:\n        lista de dataframes limpos, com linhas sincronizadas\n    """\n    base_df = dfs[0]\n    df_num = base_df.select_dtypes(include=np.number)\n    Q1 = df_num.quantile(0.25)\n    Q3 = df_num.quantile(0.75)\n    IQR = Q3 - Q1\n\n    # Máscara de linhas que não são outliers\n    mask = ~((df_num < (Q1 - k*IQR)) | (df_num > (Q3 + k*IQR))).any(axis=1)\n\n    # Aplica a mesma máscara a todos os dataframes\n    return [df.loc[mask] for df in dfs]\n\n# Uso\ndf_model, df_log, df_sqrt = remove_outliers_iqr_sync(df_model, df_log, df_sqrt)'

In [35]:
# Variáveis a plotar (todas exceto 'ros_p')
vars_to_plot = [col for col in df_model.columns if col != 'ros_p']

In [36]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import gaussian_kde
from sklearn.metrics import r2_score

# Criar PDF para salvar todos os plots
pdf_filename = '../../Data/Data_Exploration/linear_or_log_scatter_plot.pdf'
with PdfPages(pdf_filename) as pdf:

    # Check if 'ros_p' exists in all dataframes before starting the loop
    if 'ros_p' not in df_model.columns or 'ros_p' not in df_log.columns or 'ros_p' not in df_sqrt.columns:
        print("'ros_p' não encontrado em todos os shapefiles, pulando todos os plots.")
    else:
        # Loop por todas as variáveis
        for var in vars_to_plot:
            missing_vars = [df_name for df_name, df in [('Model', df_model), ('Model Log', df_log), ('Model Sqrt', df_sqrt)]
                            if var not in df.columns]
            if missing_vars:
                print(f"{var} não encontrado nos shapefiles: {', '.join(missing_vars)}, pulando.")
                continue

            print(f"\nPlotando {var} vs log(ros_p) nos shapefiles...")

            # Preparar dados
            datasets = []
            for df_name, df in [('Model', df_model), ('Model Log', df_log), ('Model Sqrt', df_sqrt)]:
                x = pd.to_numeric(df[var], errors='coerce')
                y = pd.to_numeric(df['ros_p'], errors='coerce')
                mask = (y > 0) & (x.notna()) & (y.notna())
                datasets.append((df_name, x[mask], np.log(y[mask])))

            # Criar figura lado a lado
            fig, axes = plt.subplots(1, 3, figsize=(24, 6))
            fig.suptitle(f'{var} vs log(ros_p) - Model vs Model Log vs Model Sqrt', fontsize=16, y=1.02)

            for ax, (title, x_clean, y_clean) in zip(axes, datasets):
                # Check if we have enough data
                if len(x_clean) < 5 or len(y_clean) < 5:
                    ax.text(0.5, 0.5, 'Dados insuficientes', ha='center', va='center', transform=ax.transAxes, fontsize=12)
                    ax.set_title(title)
                    continue

                try:
                    # Regressão linear
                    coeffs = np.polyfit(x_clean, y_clean, 1)
                    y_pred = np.polyval(coeffs, x_clean)
                    r2 = r2_score(y_clean, y_pred)
                    
                    # Correlação de Pearson
                    r, p_value = stats.pearsonr(x_clean, y_clean)
                    
                    # Linha de ajuste
                    x_fit = np.linspace(x_clean.min(), x_clean.max(), 300)
                    y_fit = np.polyval(coeffs, x_fit)
                    
                    # Densidade
                    xy = np.vstack([x_clean, y_clean])
                    z = gaussian_kde(xy)(xy)
                    idx = z.argsort()
                    x_sorted, y_sorted, z_sorted = x_clean.iloc[idx], y_clean.iloc[idx], z[idx]

                    # Scatter plot with density coloring
                    scatter = ax.scatter(x_sorted, y_sorted, c=z_sorted, s=20, cmap='viridis', alpha=0.6, edgecolors='none')
                    plt.colorbar(scatter, ax=ax, label='Densidade')

                    # Plot regression line
                    ax.plot(x_fit, y_fit, color='red', linewidth=2, linestyle='--',
                            label=f'y={coeffs[0]:.3f}x+{coeffs[1]:.3f}\nR²={r2:.3f}\nr={r:.3f}, p={p_value:.2e}')

                    # Set y-axis ticks with both log and linear values
                    y_min, y_max = y_clean.min(), y_clean.max()
                    y_ticks = np.linspace(y_min, y_max, 5)
                    ax.set_yticks(y_ticks)
                    
                    # Create labels showing log value and linear value in parentheses
                    new_labels = []
                    for tick in y_ticks:
                        linear_val = np.exp(tick)
                        if linear_val < 0.1:
                            linear_str = f"{linear_val:.3f}"
                        elif linear_val < 1:
                            linear_str = f"{linear_val:.2f}"
                        elif linear_val < 10:
                            linear_str = f"{linear_val:.1f}"
                        else:
                            linear_str = f"{linear_val:.0f}"
                        new_labels.append(f"{tick:.1f}\n({linear_str})")
                    ax.set_yticklabels(new_labels)
                    ax.set_ylabel('log(ros_p) [ros_p linear]', fontsize=10)

                except (ValueError, np.linalg.LinAlgError) as e:
                    ax.text(0.5, 0.5, f'Erro no cálculo: {str(e)}', ha='center', va='center', transform=ax.transAxes, fontsize=10)

                ax.set_title(title)
                ax.set_xlabel(var)
                ax.legend(fontsize=8)
                ax.grid(True, linestyle='--', alpha=0.3)

            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')  # Salvar no PDF
            plt.close(fig)

print(f"\nTodos os plots concluídos e salvos em '{pdf_filename}'")



Plotando inidoy vs log(ros_p) nos shapefiles...

Plotando enddoy vs log(ros_p) nos shapefiles...

Plotando duration_p vs log(ros_p) nos shapefiles...

Plotando 1_3y_fir_p vs log(ros_p) nos shapefiles...

Plotando 3_8y_fir_p vs log(ros_p) nos shapefiles...

Plotando 8_ny_fir_p vs log(ros_p) nos shapefiles...

Plotando elev_av vs log(ros_p) nos shapefiles...

Plotando aspect_av vs log(ros_p) nos shapefiles...

Plotando landform vs log(ros_p) nos shapefiles...

Plotando fuel_model vs log(ros_p) nos shapefiles...

Plotando f_load_av vs log(ros_p) nos shapefiles...

Plotando land_use vs log(ros_p) nos shapefiles...

Plotando CBH_m_av vs log(ros_p) nos shapefiles...

Plotando HigCC_p_av vs log(ros_p) nos shapefiles...

Plotando LowCC_p_av vs log(ros_p) nos shapefiles...

Plotando MidCC_p_av vs log(ros_p) nos shapefiles...

Plotando TotCC_p_av vs log(ros_p) nos shapefiles...

Plotando BLH_m_av vs log(ros_p) nos shapefiles...

Plotando Cape_av vs log(ros_p) nos shapefiles...

Plotando Cin_av 

In [37]:
from scipy.stats import linregress

# Criar PDF para salvar todos os plots
pdf_filename = '../../Data/Data_Exploration/linear_or_log_kde_density_plots.pdf'
with PdfPages(pdf_filename) as pdf:
    
    for var in vars_to_plot:
        missing_dfs = [name for name, df in [('Model', df_model), ('Model Log', df_log), ('Model Sqrt', df_sqrt)] 
                       if var not in df.columns or 'ros_p' not in df.columns]
        if missing_dfs:
            print(f"{var} não encontrado em todos os shapefiles: {', '.join(missing_dfs)}, pulando.")
            continue
        
        print(f"\nPlotando KDE clouds para {var} vs ros_p nos shapefiles...")

        # Preparar dados
        datasets = []
        for df_name, df in [('Linear', df_model), ('LOG', df_log), ('Sqrt', df_sqrt)]:
            x = pd.to_numeric(df[var], errors='coerce')
            y = pd.to_numeric(df['ros_p'], errors='coerce')
            mask = x.notna() & y.notna() & np.isfinite(x) & np.isfinite(y) & (y > 0)
            datasets.append((df_name, x[mask], np.log(y[mask])))

        # Criar figura com 3 subplots
        fig, axes = plt.subplots(1, 3, figsize=(21, 6))
        fig.suptitle(f'{var} vs ros_p - KDE Cloud (Linear, LOG & Sqrt)', fontsize=16, y=0.98)

        for ax, (title, x_clean, y_clean) in zip(axes, datasets):
            if len(x_clean) < 5 or np.all(x_clean == x_clean.iloc[0]) or np.all(y_clean == y_clean.iloc[0]):
                ax.text(0.5, 0.5, 'Dados insuficientes ou constantes', ha='center', va='center', transform=ax.transAxes, fontsize=12)
                ax.set_title(title, fontsize=12, fontweight='bold')
                continue

            # Estatísticas usando linregress
            slope, intercept, r_value, p_value, std_err = linregress(x_clean, y_clean)
            y_pred = intercept + slope * x_clean
            r2 = r_value**2

            # KDE
            xy = np.vstack([x_clean, y_clean])
            kde = gaussian_kde(xy)
            x_min, x_max = x_clean.min(), x_clean.max()
            y_min, y_max = y_clean.min(), y_clean.max()
            x_range = x_max - x_min
            y_range = y_max - y_min
            x_min -= 0.1 * x_range
            x_max += 0.1 * x_range
            y_min -= 0.1 * y_range
            y_max += 0.1 * y_range
            xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
            positions = np.vstack([xx.ravel(), yy.ravel()])
            z = np.reshape(kde(positions).T, xx.shape)
            contourf = ax.contourf(xx, yy, z, levels=20, cmap='viridis', alpha=0.8)
            plt.colorbar(contourf, ax=ax, label='Densidade')

            # Ajustar ticks do eixo y
            y_ticks = ax.get_yticks()
            valid_ticks = [tick for tick in y_ticks if y_min <= tick <= y_max] or np.linspace(y_min, y_max, 5)
            ax.yaxis.set_major_locator(FixedLocator(valid_ticks))

            # Labels log e linear
            new_labels = []
            for tick in valid_ticks:
                linear_val = np.exp(tick)
                if linear_val < 0.1:
                    linear_str = f"{linear_val:.3f}"
                elif linear_val < 1:
                    linear_str = f"{linear_val:.2f}"
                elif linear_val < 10:
                    linear_str = f"{linear_val:.1f}"
                else:
                    linear_str = f"{linear_val:.0f}"
                new_labels.append(f"{tick:.1f}\n({linear_str})")
            ax.set_yticklabels(new_labels)
            ax.set_ylabel('log(ros_p) [ros_p linear]', fontsize=10)

            # Estatísticas como texto
            textstr = f'R² = {r2:.3f}\nr = {r_value:.3f}\np = {p_value:.2e}'
            ax.text(0.05, 0.95, textstr, transform=ax.transAxes,
                    fontsize=9, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

            ax.set_title(title, fontsize=12, fontweight='bold')
            ax.set_xlabel(var)
            ax.grid(True, linestyle='--', alpha=0.3)

        plt.tight_layout()
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print(f"\nTodos os KDE cloud plots concluídos e salvos em '{pdf_filename}'")



Plotando KDE clouds para inidoy vs ros_p nos shapefiles...

Plotando KDE clouds para enddoy vs ros_p nos shapefiles...

Plotando KDE clouds para duration_p vs ros_p nos shapefiles...

Plotando KDE clouds para 1_3y_fir_p vs ros_p nos shapefiles...

Plotando KDE clouds para 3_8y_fir_p vs ros_p nos shapefiles...

Plotando KDE clouds para 8_ny_fir_p vs ros_p nos shapefiles...

Plotando KDE clouds para elev_av vs ros_p nos shapefiles...

Plotando KDE clouds para aspect_av vs ros_p nos shapefiles...

Plotando KDE clouds para landform vs ros_p nos shapefiles...

Plotando KDE clouds para fuel_model vs ros_p nos shapefiles...

Plotando KDE clouds para f_load_av vs ros_p nos shapefiles...

Plotando KDE clouds para land_use vs ros_p nos shapefiles...

Plotando KDE clouds para CBH_m_av vs ros_p nos shapefiles...

Plotando KDE clouds para HigCC_p_av vs ros_p nos shapefiles...

Plotando KDE clouds para LowCC_p_av vs ros_p nos shapefiles...

Plotando KDE clouds para MidCC_p_av vs ros_p nos shapefile